In [222]:
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import requests
from requests import Session, request, get
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
from functools import reduce ## 추가 ##

In [3]:
from functools import reduce ## 추가 ##

In [4]:
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.52'}


In [5]:
major_class_dict = {'001' : '상의', '002' : '아우터', '003' : '바지', '020' : '원피스', '022' : '스커트', '018' : '스니커즈', '005' : '신발', '004' : '가방', '054' : '여성 가방', '017' : '스포츠용품', '007' : '모자'}


In [20]:
#### 최종 #####
### 상품코드 하나당 실행 ###

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?&sort=comment_cnt_desc"
params = {
    'page' : None,
    'similarNo' : None, # goodsNo와 동일
#     'goodsNo' : None,
#     'selectedSimilarNo' : None # 색상 아닌 전체
    
}
musinsa_cmt_list = list()
review_no_list = list()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()

for i in range(200):
    params['page'] = i+1
    params['goodsNo'] = 947067
#     params['similarNo'] = 947067
#     params['selectedSimilarNo'] = 0
    res = requests.get(url_good, params = params, headers=header)
    soup = BeautifulSoup(res.text, "html.parser")

    # 페이지마다 모두 뽑기
    comments = soup.find_all(attrs={"class":"comments comments-input"})
    reviews = soup.find_all(attrs={"class":"review-contents"})
    review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
    review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
    

    # 리뷰 넘버를 뽑는다
    for idx,review in enumerate(reviews):
        review_no = review["data-review-no"]
        review_no_list.append(review_no)
        
        # 리뷰 데이트를 뽑는다    
        for idx_date, date in enumerate(review_dates):
            if idx != idx_date:
                pass
            elif idx == idx_date:
                if len(date.text.split('.')) ==3:
                    review_date = date.text
                else:
                    review_date = "며칠전으로 표시된 리뷰"
        review_date_list.append(review_date)
        review_date_dict[review_no] = review_date
        
        for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
            if comment["c_idx"] == review_no:
                if comment.find("span", attrs={"class" : "cmt_summary"}) is not None:
                    verifying_text = comment.find("span", attrs={"class" : "cmt_summary"}).text
                    p = re.compile("안녕하세요\D+무신사\D+")
                    o = re.compile("지급해드렸습니다")
                    if p.search(verifying_text) is not None:
                        if o.search(verifying_text) is None:
                            if reduce(lambda x,y: x+y, review_no.split('.'))\
                                > reduce(lambda x,y: x+y, comment.find("span", attrs={"class":"recomment_date"}).text[:10].split('.')):
                                print(verifying_text, review_no, review_date)
                                recomment_text = comment.find("span", attrs={"class":"cmt_detail"}).text
                                musinsa_cmt_list.append(recomment_text)
                                review_recomment_dict[review_no] = recomment_text
            
        
        # 리뷰 사진을 뽑는다
        for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
            review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
            review_image_url_list = list()
            if idx != idx_img:
                pass
            elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                    if review_img_pre.find('img')["src"].startswith("//"):
                        review_img_url = "https:" + review_img_pre.find('img')["src"][:-5]
                    review_image_url_list.append(review_img_url)
#                 print(review_no, review_image_url_list)
                review_image_dict[review_no] = review_image_url_list

    

                       

print(review_image_dict)
print(review_date_dict)
print(review_recomment_dict)


안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 22961549 2021.12.05
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 22830205 2021.12.03
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 22645244 2021.11.30
안녕하세요. 무신사 스토어 고객센터입니다. 많이 기대하시고 주문하셨을 텐데 상품 상태로 인해 불편을 안겨드려 대단히 죄송합니다.상품 출고 준비 시 보다 철저한 검수 과정을 거쳐... 22302492 2021.11.24
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 22230525 2021.11.22
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 21997506 2021.11.16
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 21877078 2021.11.13
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 20451498 2021.10.07
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한

안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6511197 2019.10.02
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6494750 2019.10.01
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6492255 2019.10.01
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6445197 2019.09.28
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6444655 2019.09.28
안녕하세요. 무신사 스토어 고객센터입니다.죄송스럽지만, 현재 작성해주신 구매 후기는 조건 미충족으로 인하여 적립금 지급이 보류된 상태입니다.해당 부분 도움을 드리기 위해 1... 6418987 2019.09.27
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6409567 2019.09.26
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.▶ ... 6391777 2019.09.25
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러

안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 20933378 2021.10.21
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우... 20728839 2021.10.15
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 스타일 후기 적립금 지급 기준이 충족되지 않아 지급이 보류되었습니다.수고로우시겠... 20629221 2021.10.13
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다. 그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다... 9741503 2020.05.19
안녕하세요. 무신사 스토어 고객센터입니다.정성스럽게 올려주신 소중한 후기 감사합니다. 그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.... 9709544 2020.05.17
안녕하세요. 무신사 스토어 고객센터입니다.정성스럽게 올려주신 소중한 후기 감사합니다. 그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.... 9705643 2020.05.16
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.... 9450310 2020.05.04
안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.... 9442357 2020.05.03
안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 

In [ ]:
#### 코랩, 불안정코드 ### 
### 상품코드 하나당 실행 ###

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?&sort=comment_cnt_desc"
params = {
    'page' : None,
    # 'similarNo' : None, # goodsNo와 동일
    'goodsNo' : None,
    # 'selectedSimilarNo' : None # 색상 아닌 전체
    
}
musinsa_cmt_list = list()
review_no_list = list()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()


for goods_no in new_goods_list:
    for i in [0]:
        params['page'] = i+1
        params['goodsNo'] = goods_no
      # params['similarNo'] = goods_no
      # params['selectedSimilarNo'] = 0
        res = requests.get(url_good, params = params, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

      # 페이지마다 모두 뽑기
        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
        review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
      

      # 리뷰 넘버를 뽑는다
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            review_no_list.append(review_no)

          # 리뷰 데이트를 뽑는다    
          for idx_date, date in enumerate(review_dates):
            if idx != idx_date:
                pass
            elif idx == idx_date:
                if len(date.text.split('.')) ==3:
                    review_date = date.text
                else:
                    review_date = "며칠전으로 표시된 리뷰"
                review_date_list.append(review_date)
                review_date_dict[review_no] = review_date
          
          for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
                if comment["c_idx"] == review_no:
                    verifying_location = comment.find("span", attrs={"class" : "cmt_summary"})
                    if verifying_location is not None:
                        verifying_text = verifying_location.text
                    p = re.compile("안녕하세요\D+무신사\D+")
                    if p.search(verifying_text) is not None:
                        if reduce(lambda x,y: x+y, review_no.split('.'))\
                          >= reduce(lambda x,y: x+y, comment.find("span", attrs={"class":"recomment_date"}).text[:10].split('.')):
  #                         print(verifying_text, review_no)
                          musinsa_cmt_list.append(verifying_text)
                          review_recomment_dict[review_no] = verifying_text
              
          
          # 리뷰 사진을 뽑는다
          for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
              review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
              review_image_url_list = list()
              if idx != idx_img:
                  pass
              elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                  for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                      if review_img_pre.find('img')["src"].startswith("//"):
                          review_img_url = "https:" + review_img_pre.find('img')["src"][:-5]
                      review_image_url_list.append(review_img_url)
  #                 print(review_no, review_image_url_list)
                  review_image_dict[review_no] = review_image_url_list

                os.chdir("/content/drive/MyDrive/1st project/{}/{}/{}".format([goodsNo:3], goodsNo, goods_no)) # 경로 설정
                os.makedirs("./style", exist_ok = True)
                os.chdir("./style")
                with open("{}_style_from_page{}-{}.jpg".format(goodsNo, page, idx+1), "wb") as f:
                      f.write(image_res.content)
                      f.close()

                       

print(review_image_dict)
print(review_date_dict)
print(review_recomment_dict)


In [22]:
set(musinsa_cmt_list), len(set(musinsa_cmt_list)), len(musinsa_cmt_list), len(review_no_list), len(review_date_list), le

({'안녕하세요. 무신사 스토어 고객센터입니다. 구매하신 상품에 대해 아쉬운 점을 안겨드리게 되어 저희 또한 아쉬운 마음입니다. 앞으로 더욱 만족스러운 쇼핑이 될 수 있도록 끊임없이 노력하는 무신사 스토어가 되겠습니다. 고객님의 소중한 구매 후기 감사드립니다. 감사합니다.',
  '안녕하세요. 무신사 스토어 고객센터입니다. 많이 기대하시고 주문하셨을 텐데 상품 상태로 인해 불편을 안겨드려 대단히 죄송합니다.상품 출고 준비 시 보다 철저한 검수 과정을 거쳐서 발송될 수 있도록 더욱 노력하겠습니다.혹여 상품에 대해 다른 불편사항이나 도움이 필요하시다면 고객센터로 문의 부탁드립니다.고객님의 편안한 쇼핑을 위하여 최선을 다하는 무신사 스토어가 되겠습니다. ※ 해당 후기는 정상적으로 노출됩니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.',
  '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다. 그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.▶ 스타일 후기 기준스타일 후기는 제품을 구매하신 분의 전신사진을 등록해 주시는 후기로, 제품을 구매한 고객의 패션 스타일을 다른 구매 고객이 참고할 수 있도록 하는 것을 목적으로 합니다. ① 구매하신 상품을 착용한 전신 스타일링 컷(얼굴 제외 가능, 어깨부터 발끝까지 프레임 안에 전부 보이도록 촬영한 사진)② 착용이 어려운 상품은 전신 스타일링한 상태에서 제품이 보이도록 든 상태의 사진③ 20자 이상의 상품에 대한 후기(특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.) 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메뉴(≡) → 구매 후기 → 후기 내역 → 수정 희망하시는 후기 내용 클릭 → 수정② PC : 무신사 홈페이지 → 로그인 → 마이페이지

In [24]:
len(review_image_dict.keys())

2000

In [6]:
import sqlite3

In [7]:
con = sqlite3.connect("./musinsa (3).db")
sql = "SELECT goods_id FROM 리뷰개수테이블 WHERE stylereviewCount >= 50 AND photoreviewCount >= 50;"
cur = con.cursor()
cur.execute(sql)
rows = cur.fetchall()
con.close()


In [8]:
goodcode_list = list()
for i in rows:
    goodcode_list.append(i[0])

In [99]:
### 11.01 수정 ###

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?&sort=comment_cnt_desc"
params = {
    'page' : None,
    'goodsNo' : None
    #'similarNo' : None, # goodsNo와 동일
    # 'selectedSimilarNo' : None # 색상 아닌 전체
    }

musinsa_cmt_list = list()
review_no_list = list()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()
goods_list = list()
review_image_num_dict = dict()


for goodsNo in goodcode_list[:10]:
    for i in range(200): # 페이지 range을 수정해야한다. 
        params['page'] = i+1
        params['goodsNo'] = goodsNo
        # params['similarNo'] = goodsNo
        # params['selectedSimilarNo'] = 0
        res = requests.get(url_good, params = params, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

            # 페이지마다 모두 뽑기
        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
        review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
#         print("상품", goodsNo)
            

            # 리뷰 넘버를 뽑는다
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            review_no_list.append(review_no)
            goods_list.append(goodsNo)
#             print("리뷰넘버 :", review_no)
                # 리뷰 데이트를 뽑는다    
            for idx_date, date in enumerate(review_dates):
                if idx != idx_date:
                    pass
                elif idx == idx_date:
                    if len(date.text.split('.')) ==3:
                        review_date = date.text
                    else:
                        review_date = "며칠전으로 표시된 리뷰"
            review_date_list.append(review_date)
            review_date_dict[review_no] = review_date

                
            for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
#                 print(idx_cmt)
                if idx != idx_cmt:
                    pass
                elif idx == idx_cmt:
                    find_recomments = comment.find_all("div", attrs={"class":"comment_recomment comment_wrap"}, recursive=True)
                    for idx_rcmt, find_recomment in enumerate(find_recomments):
#                         print("답글번호", idx_rcmt, find_recomment.find("span", attrs={"class":"cmt_summary"}))
                        if find_recomment.find("img", attrs={"alt":"무신사 관리자 썸네일"}) is not None: # 무신사가 답글 단 경우 
                            recomment_sum = find_recomment.find("span", attrs={"class":"cmt_summary"})
                            recomment_date = find_recomment.find("span", attrs={"class":"recomment_date"})
                            recomment_detail = find_recomment.find("span", attrs={"class":"cmt_detail"})
#                             print(recomment_sum, recomment_date, recomment_detail)
#                             print(review_date_dict[comment["c_idx"]].split('.'), recomment_date.text[:10].split('.'))
 
                            if reduce(lambda x,y: x+y, review_date_dict[comment["c_idx"]].split('.'))\
                                        > reduce(lambda x,y: x+y, recomment_date.text[:10].split('.')): # 리뷰날짜가 답글날짜보다 최신이면 정상
                                pass
                            else:
                                if recomment_detail:
                                    verifying_text = recomment_detail.text
                                    p = re.compile("안녕하세요\D+무신사\D+")
                                    o = re.compile("지급해드렸습니다")
                                    if p.search(verifying_text) is not None:
                                        if o.search(verifying_text) is None: # 지급해드렸습니다 답글 걸러냄
#                                             print(review_date_dict[comment["c_idx"]], recomment_date.text[:10], recomment_detail.text)

                                            musinsa_cmt_list.append(recomment_detail.text)
                                            review_recomment_dict[review_no] = recomment_detail.text

#             print(len(review_recomment_dict.keys()) )

                    
             # 리뷰 사진을 뽑는다
            for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
                review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
                review_image_url_list = list()
                if idx != idx_img:
                    pass
                elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                    for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                        if review_img_pre.find('img')["src"].startswith("//"):
                            review_img_url = "https:" + review_img_pre.find('img')["src"][:-5]
                        review_image_url_list.append(review_img_url)
        #                 print(review_no, review_image_url_list)
                    review_image_dict[review_no] = str(review_image_url_list)

                    review_image_num_dict[review_no] = len(review_image_url_list)
                


print(len(review_image_dict.keys()))

print(len(review_date_dict.keys()))

print(len(review_recomment_dict.keys()))

# print(review_image_dict)

# print(review_date_dict)

# print(review_recomment_dict.keys())

# print("상품", goodsNo)

15980
15982
1949


In [100]:
print(len(review_image_dict.keys()))

print(len(review_date_dict.keys()))

print(len(review_recomment_dict.keys()))

len(set(musinsa_cmt_list))

15980
15982
1949


159

In [101]:
set(musinsa_cmt_list)

{'"안녕하세요. 무신사 스토어 후기 담당자입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류상태입니다.1) 미착용 상태의 단순 제품만 촬영 (스타일 후기와 중복 사용 방지)2) 상품에 대해 식별이 가능해야 함 & 상품 전체 컷으로 촬영한 사진 이여야 함3) 상품이 미포장 상태 이여야 함4) 20자 이상의 내용번거로우시더라도, 위 기준을 참고하여 수정 부탁드리겠습니다.감사합니다. 언제나 행복 가득한 하루 보내세요!"',
 '안녕하세요. 무신사 스토어 고객센터입니다. 구매하신 상품에 대해 아쉬운 점을 안겨드리게 되어 저희 또한 아쉬운 마음입니다. 앞으로 더욱 만족스러운 쇼핑이 될 수 있도록 끊임없이 노력하는 무신사 스토어가 되겠습니다. 고객님의 소중한 구매 후기 감사드립니다. 감사합니다.',
 '안녕하세요. 무신사 스토어 고객센터입니다. 구매하신 상품에 대해 아쉬운 점을 안겨드리게 되어 저희 또한 아쉬운 마음입니다. 죄송스럽게도 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. ▶ 스타일 후기 적립금 지급 조건 ① 반드시 구매하신 상품을 착용한 전신 스타일링 컷② 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷③ 20자 이상의 상품에 대한 후기(텍스트)TIP. 얼굴은 제외 가능하나, 어깨부터 발끝까지는 반드시 프레임 안에 담겨야 합니다.신체의 일부만 등록되거나, 제품 식별이 어려운 경우에는 지급이 보류될 수 있습니다. 특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 마이페이지 → 후기 작성 → 후기 내역 → 수정 희망 하시는 후기 내용 클릭 → 수정② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 구매 후기 → 후기 내역 → 수정참고로 후기 

In [98]:
set(musinsa_cmt_list)

{'안녕하세요. 무신사 스토어 고객센터입니다. 구매하신 상품에 대해 아쉬운 점을 안겨드리게 되어 저희 또한 아쉬운 마음입니다. 앞으로 더욱 만족스러운 쇼핑이 될 수 있도록 끊임없이 노력하는 무신사 스토어가 되겠습니다. 고객님의 소중한 구매 후기 감사드립니다. 감사합니다.',
 '안녕하세요. 무신사 스토어 고객센터입니다. 많이 기대하시고 주문하셨을 텐데 상품 상태로 인해 불편을 안겨드려 대단히 죄송합니다.상품 출고 준비 시 보다 철저한 검수 과정을 거쳐서 발송될 수 있도록 더욱 노력하겠습니다.혹여 상품에 대해 다른 불편사항이나 도움이 필요하시다면 고객센터로 문의 부탁드립니다.고객님의 편안한 쇼핑을 위하여 최선을 다하는 무신사 스토어가 되겠습니다. ※ 해당 후기는 정상적으로 노출됩니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.',
 '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다. 그러나, 아래와 같이 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.▶ 스타일 후기 기준스타일 후기는 제품을 구매하신 분의 전신사진을 등록해 주시는 후기로, 제품을 구매한 고객의 패션 스타일을 다른 구매 고객이 참고할 수 있도록 하는 것을 목적으로 합니다. ① 구매하신 상품을 착용한 전신 스타일링 컷(얼굴 제외 가능, 어깨부터 발끝까지 프레임 안에 전부 보이도록 촬영한 사진)② 착용이 어려운 상품은 전신 스타일링한 상태에서 제품이 보이도록 든 상태의 사진③ 20자 이상의 상품에 대한 후기(특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.) 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메뉴(≡) → 구매 후기 → 후기 내역 → 수정 희망하시는 후기 내용 클릭 → 수정② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 

In [89]:
set(musinsa_cmt_list), len(set(musinsa_cmt_list))

({"안녕하세요. 무신사 스토어 고객센터입니다. 먼저, 이용에 불편을 드려 대단히 죄송합니다. 해당 내용을 1:1 문의 게시판으로 접수해 드렸습니다. 1:1문의를 확인해 주시면 감사하겠습니다.더불어, 현재 작성해 주신 후기는 죄송스럽게도 '조건 미충족' 으로 인하여 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.지급 보류 유형 9.상품 미착용 컷▶ 지급 보류 상세사유 : 구매하신 상품을 착용한 전신 스타일링 컷으로 수정을 부탁드립니다.TIP. 얼굴은 제외 가능하며, 어깨부터 발끝까지 프레임 안에 담겨야 합니다.착용이 어려운 상품은 휴대 상태 전신 컷 또는 전신 스타일링 컷 1장, 구매하신 상품 단독 컷 1장으로 올려주시면 적립금 지급이 가능합니다.(예:이너웨어, 화장품, 스마트기기, 옷걸이 등등)특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 스타일 후기 적립금 지급 조건 - 반드시 구매하신 상품을 착용한 전신 스타일링 컷- 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷 - 20자 이상의 상품에 대한 후기 ▶ 상품평 수정 방법- PC, APP :  마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 후기 적립금은 자동 지급이 아닌, 수기로 지급되며 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.※ 해당 후기는 정상적으로 노출됩니다.",
  '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. ▶ 스타일 후기 적립금 지급 조건

In [12]:
con = sqlite3.connect("C:\\2022 SWA 아카데미\\1차프로젝트\\musinsa style review.db")
sql = "SELECT review_recomment FROM 리뷰테이블"
cur = con.cursor()
cur.execute(sql)
review_recomment_list = cur.fetchall()
con.close()


In [14]:
len(review_recomment_list)

124682

In [16]:
len(set(review_recomment_list))

142

In [18]:
set(review_recomment_list)

{("안녕하세요. 무신사 스토어 고객센터입니다. 구매하신 상품에 대해 아쉬운 점을 안겨드리게 되어 저희 또한 아쉬운 마음입니다.보다 더 만족스러운 상품으로 제공될 수 있도록 끊임없이 노력하겠습니다.혹여 상품에 대해 다른 불편사항이나 도움이 필요하시다면 고객센터로 문의 부탁드립니다고객님의 깊은 양해 부탁드립니다. 더불어, 현재 작성해 주신 후기는 죄송스럽게도 '조건 미충족' 으로 인하여 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.지급 보류 유형 8.전신 실루엣 식별 어려운 컷▶ 지급 보류 상세 사유 : 신체 일부가 가려진 상태의 사진 컷 사용TIP. 얼굴은 제외 가능하며, 어깨부터 발끝까지 프레임 안에 담겨야 합니다.착용이 어려운 상품은 휴대 상태 전신 컷 또는 전신 스타일링 컷 1장과 구매하신 상품 단독 컷 1장으로 올려주시면 적립금 지급이 가능합니다.(예:이너웨어, 화장품, 스마트기기, 옷걸이 등등)특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 스타일 후기 적립금 지급 조건 - 반드시 구매하신 상품을 착용한 전신 스타일링 컷- 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷 - 20자 이상의 상품에 대한 후기 ▶ 상품평 수정 방법- PC, APP :  마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 후기 적립금은 자동 지급이 아닌, 수기로 지급되며 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.※ 해당 후기는 정상적으로 노출됩니다.※ 댓글은 관리자가 확인하지 않습니다. 후기 관련하여 궁금하신 내용이 있으실 경우에는 1:1문의로 문의 부탁드립니다.",),
 ('안녕하세요 무신사 스토어 후기 담당자 승식 입니다:) 고객님의 개성 넘치는 스타일 후기가 확인 완료 되었습니다. 다른 분들께서도 고객님의 센스 있는 스타일을 참고하실 수 있을 것 같습니

In [102]:
### 11.01 수정 ###
### 상품후기 ### 

url_good =  "https://goods.musinsa.com/api/goods/v1/review/photo/list?&sort=comment_cnt_desc"
params = {
    'page' : None,
    'goodsNo' : None
    #'similarNo' : None, # goodsNo와 동일
    # 'selectedSimilarNo' : None # 색상 아닌 전체
    }

musinsa_cmt_list = list()
review_no_list = list()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()
goods_list = list()
review_image_num_dict = dict()


for goodsNo in goodcode_list[:2]:
    for i in range(5): # 페이지 range을 수정해야한다. 
        params['page'] = i+1
        params['goodsNo'] = goodsNo
        # params['similarNo'] = goodsNo
        # params['selectedSimilarNo'] = 0
        res = requests.get(url_good, params = params, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

            # 페이지마다 모두 뽑기
        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
        review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
#         print("상품", goodsNo)
            

            # 리뷰 넘버를 뽑는다
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            review_no_list.append(review_no)
            goods_list.append(goodsNo)
#             print("리뷰넘버 :", review_no)
                # 리뷰 데이트를 뽑는다    
            for idx_date, date in enumerate(review_dates):
                if idx != idx_date:
                    pass
                elif idx == idx_date:
                    if len(date.text.split('.')) ==3:
                        review_date = date.text
                    else:
                        review_date = "며칠전으로 표시된 리뷰"
            review_date_list.append(review_date)
            review_date_dict[review_no] = review_date

                
            for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
#                 print(idx_cmt)
                if idx != idx_cmt:
                    pass
                elif idx == idx_cmt:
                    find_recomments = comment.find_all("div", attrs={"class":"comment_recomment comment_wrap"}, recursive=True)
                    for idx_rcmt, find_recomment in enumerate(find_recomments):
#                         print("답글번호", idx_rcmt, find_recomment.find("span", attrs={"class":"cmt_summary"}))
                        if find_recomment.find("img", attrs={"alt":"무신사 관리자 썸네일"}) is not None: # 무신사가 답글 단 경우 
                            recomment_sum = find_recomment.find("span", attrs={"class":"cmt_summary"})
                            recomment_date = find_recomment.find("span", attrs={"class":"recomment_date"})
                            recomment_detail = find_recomment.find("span", attrs={"class":"cmt_detail"})
#                             print(recomment_sum, recomment_date, recomment_detail)
#                             print(review_date_dict[comment["c_idx"]].split('.'), recomment_date.text[:10].split('.'))
 
                            if reduce(lambda x,y: x+y, review_date_dict[comment["c_idx"]].split('.'))\
                                        > reduce(lambda x,y: x+y, recomment_date.text[:10].split('.')): # 리뷰날짜가 답글날짜보다 최신이면 정상
                                pass
                            else:
                                if recomment_detail:
                                    verifying_text = recomment_detail.text
                                    p = re.compile("안녕하세요\D+무신사\D+")
                                    o = re.compile("지급해드렸습니다")
                                    if p.search(verifying_text) is not None:
                                        if o.search(verifying_text) is None: # 지급해드렸습니다 답글 걸러냄
#                                             print(review_date_dict[comment["c_idx"]], recomment_date.text[:10], recomment_detail.text)

                                            musinsa_cmt_list.append(recomment_detail.text)
                                            review_recomment_dict[review_no] = recomment_detail.text

#             print(len(review_recomment_dict.keys()) )

                    
             # 리뷰 사진을 뽑는다
            for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
                review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
                review_image_url_list = list()
                if idx != idx_img:
                    pass
                elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                    for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                        if review_img_pre.find('img')["src"].startswith("//"):
                            review_img_url = "https:" + review_img_pre.find('img')["src"][:-5]
                        review_image_url_list.append(review_img_url)
        #                 print(review_no, review_image_url_list)
                    review_image_dict[review_no] = str(review_image_url_list)

                    review_image_num_dict[review_no] = len(review_image_url_list)
                


print(len(review_image_dict.keys()))

print(len(review_date_dict.keys()))

print(len(review_recomment_dict.keys()))

# print(review_image_dict)

# print(review_date_dict)

# print(review_recomment_dict.keys())

# print("상품", goodsNo)

100
100
88


In [111]:
musinsa_cmt_list, len(set(musinsa_cmt_list)),  len(musinsa_cmt_list)

(['안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류 상태입니다.▶ 포토 후기 기준① 구매하신 상품을 착용하지 않은 상태로 해당 상품만 촬영한 사진(스타일 후기와 중복 사용 방지) ② 겉 포장을 제거하여 접힌 부분을 펼쳐서 상품의 전체 실루엣으로 촬영한 사진(사진 등록 시 상품 전체 실루엣이 프레임 안에 전부 보여야 함)③ 세트, 한 쌍의 상품일 경우 함께 전부 보이도록 촬영한 사진(액세서리, 레그웨어/속옷, 신발 등)④ 20자 이상의 상품에 대한 후기(특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.) 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메뉴(≡) → 구매 후기 → 후기 내역 → 수정 희망하시는 후기 내용 클릭 → 수정② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 구매 후기 → 후기 내역 → 수정감사합니다.',
  '안녕하세요. 무신사 스토어 고객센터입니다.고객님께서 정성스럽게 올려주신 소중한 후기 감사합니다.그러나, 아래 적립금 지급 기준과 부적합하여 적립금 지급 보류 상태입니다.▶ 포토 후기 기준① 구매하신 상품을 착용하지 않은 상태로 해당 상품만 촬영한 사진(스타일 후기와 중복 사용 방지) ② 겉 포장을 제거하여 접힌 부분을 펼쳐서 상품의 전체 실루엣으로 촬영한 사진(사진 등록 시 상품 전체 실루엣이 프레임 안에 전부 보여야 함)③ 세트, 한 쌍의 상품일 경우 함께 전부 보이도록 촬영한 사진(액세서리, 레그웨어/속옷, 신발 등)④ 20자 이상의 상품에 대한 후기(특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.) 수고로우시겠지만, 위 기준을 참고하여 수정해주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 메

In [109]:
p = re.compile("(지급 보류 사유).+[^0-9a-zA-Zㄱ-ㅎ가-힣]|(지급 보류 상세 사유).+[^0-9a-zA-Zㄱ-ㅎ가-힣]")
re.search(p, '안녕하세요. 무신사 스토어 고객센터입니다.정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다.▶ 지급 보류 사유 : 신체의 일부만 등록되거나 상품 식별이 어려운 컷 사용 ▶ 상품 사진 후기 적립금 지급 조건① 겉 포장을 제거하여 접힌 부분을 펼쳐서 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진② 20자 이상의 상품에 대한 후기TIP. 상품 사진 후기는 상품을 정밀하게 확인하실 수 있도록 만든 후기로 신체의 일부만 등록된 경우에는 상품 단독 컷을 추가하면 정상 지급 가능합니다. 특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 상품평 수정 방법① 모바일 : 앱 로그인 → 마이페이지 → 후기 작성 → 후기 내역 → 수정 희망 하시는 후기 내용 클릭 → 수정② PC : 무신사 홈페이지 → 로그인 → 마이페이지 → 구매 후기 → 후기 내역 → 수정후기 적립금은 자동 지급이 아닌, 수기로 순차적으로 지급을 도와드리고 있습니다.수정 후 영업일 기준 1~2일 정도 소요되는 점 양해 부탁드립니다.')

<re.Match object; span=(149, 625), match='지급 보류 사유 : 신체의 일부만 등록되거나 상품 식별이 어려운 컷 사용 ▶ 상품 사진 >

In [110]:
p = re.compile("(지급 보류 사유).+[^0-9a-zA-Zㄱ-ㅎ가-힣]|(지급 보류 상세 사유).+[^0-9a-zA-Zㄱ-ㅎ가-힣]")
re_list = list()
for i in range(len(musinsa_cmt_list)):
    re_list.append(re.search(p,musinsa_cmt_list[i]))

print(re_list)

[None, None, None, None, None, None, None, None, None, None, None, <re.Match object; span=(174, 862), match='지급 보류 상세 사유 : 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진으>, <re.Match object; span=(177, 751), match='지급 보류 상세 사유 : 신체의 일부만 등록 (상품만을 촬영한 컷 포함) 상품만을 촬영한>, <re.Match object; span=(184, 758), match='지급 보류 상세 사유 : 상품의 형태가 프레임 내에 또렷하게 보이도록 직접 촬영한 사진으>, <re.Match object; span=(169, 797), match='지급 보류 상세 사유 : 크루 넥 긴팔 티셔츠 [블루 그레이] 상품의 후기 사진과 동일한>, <re.Match object; span=(169, 860), match='지급 보류 상세 사유 : 구매 후기 상품과 사진 속 상품이 상이한 것으로 확인됩니다. ->, <re.Match object; span=(169, 860), match='지급 보류 상세 사유 : 구매 후기 상품과 사진 속 상품이 상이한 것으로 확인됩니다. ->, <re.Match object; span=(170, 797), match='지급 보류 상세 사유 : 신체의 일부만 등록되거나 상품 식별이 어려운 컷 사용 등록해 주>, <re.Match object; span=(170, 864), match='지급 보류 상세 사유 : 구매 후기 상품과 사진 속 상품이 상이한 것으로 확인됩니다. ->, <re.Match object; span=(151, 698), match='지급 보류 사유 : 프레임안에 상품의 전체 실루엣이 나오지 않은 상태의 사진 컷 사용(소>, <re.Match object; span=(151, 698), match='지급 보류 사유 : 프레임안에 상품의 전체 실루엣이 나오지 않은 상태의 사진 컷 

In [25]:
str(review_recomment_list[30])

"('적합',)"

In [20]:
p = re.compile("지급 보류 사유")

In [22]:
for i in set(review_recomment_list):
    re.search(str(i), p)

TypeError: expected string or bytes-like object

In [40]:
con = sqlite3.connect("./musinsa (3).db")
sql = "SELECT major_category, mid_category, goods_id FROM 소분류테이블"
cur = con.cursor()
cur.execute(sql)
rows = cur.fetchall()
con.close()

In [42]:
len(rows)

34434

In [ ]:
for n in range(len(rows)):
    os.chdir("./1st project2")
    os.makedirs("./{}/{}/{}/style".format(rows[n][0], rows[n][1]), rows[n][2], exist_ok=True) # 경로 설정
    os.makedirs("./{}/{}/{}/photo".format(rows[n][0], rows[n][1]), rows[n][2], exist_ok=True)

In [43]:
for n in range(len(rows)):
# for mdcls in mid_class_list_bymajorclass: # 대분류 선택 020 022
#     for k in mdcls: # 중분류 선택 
        # url_midclass =  "https://www.musinsa.com/categories/item/" + k  
        # # 중분류 별 상품 코드, 
        # # goodcode_list = list()
        # for page in range(1,10):

        #     url_midclasses = url_midclass + "?sort=emt_high&page=" + str(page)
            
        #     res = requests.get(url_midclasses,headers=header,verify = False)
        #     soup = BeautifulSoup(res.text, "html.parser")

        #     images = soup.find_all("img", attrs={"class":"lazyload lazy"}) # 한 페이지, 90개 상품

            # print(images)
            # for idx,image in enumerate(images): # idx : max90
                os.makedirs("./1st project2/{}/{}".format(rows[n][0], rows[n][1]), exist_ok=True) # 경로 설정
                os.chdir("./1st project2/{}/{}".format(rows[n][0], rows[n][1])) # 경로 설정

#                     #print(image['src'])
                # print(idx, image)
                # image_url = image["data-original"]
                # if image_url.startswith("/"):
                #     image_url = "https:" + image_url

                # image_res = requests.get(image_url)
                # image_res.raise_for_status()


                # goodcode_url = image.find_parent()['href'] 
                # goodcode = goodcode_url.split('/')[-1] # 상품코드
                # goodcode_list.append(goodcode)
                
                # major_category.append(k[:3])
                # mid_category.append(k)
                # goods_id.append(goodcode)
                # goods_image_url.append(image_url)

                # os.makedirs("./{}/sample".format(goodcode), exist_ok = True)
                os.makedirs("./{}/style".format(rows[n][2]), exist_ok = True)
                os.makedirs("./{}/photo".format(rows[n][2]), exist_ok = True)
                # os.chdir("./{}/sample".format(goodcode))
                # print(os.getcwd())
                # with open("{}_sample_from_page{}-{}.jpg".format(goodcode, page, idx+1), "wb") as f:
                #       f.write(image_res.content)
                #       f.close()

#                 if idx>=89:
#                     break
        
#         goodcode_mid_dict[k] = goodcode_list
    
#     list1.append(goodcode_mid_dict)

# print(list1)

KeyboardInterrupt: 

In [31]:
#### 최종 #####
### 상품코드 하나당 실행 ###

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list?&sort=comment_cnt_desc"
params = {
    'page' : None,
    'similarNo' : None, # goodsNo와 동일
#     'goodsNo' : None,
#     'selectedSimilarNo' : None # 색상 아닌 전체
    
}
musinsa_cmt_list = list()
review_no_list = list()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()

for i in range(200):
    params['page'] = i+1
    params['goodsNo'] = 1446775
#     params['similarNo'] = 947067
#     params['selectedSimilarNo'] = 0
    res = requests.get(url_good, params = params, headers=header)
    soup = BeautifulSoup(res.text, "html.parser")

    # 페이지마다 모두 뽑기
    comments = soup.find_all(attrs={"class":"comments comments-input"})
    reviews = soup.find_all(attrs={"class":"review-contents"})
    review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
    review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
    

    # 리뷰 넘버를 뽑는다
    for idx,review in enumerate(reviews):
        review_no = review["data-review-no"]
        review_no_list.append(review_no)
        
        # 리뷰 데이트를 뽑는다    
        for idx_date, date in enumerate(review_dates):
            if idx != idx_date:
                pass
            elif idx == idx_date:
                if len(date.text.split('.')) ==3:
                    review_date = date.text
                else:
                    review_date = "며칠전으로 표시된 리뷰"
        review_date_list.append(review_date)
        review_date_dict[review_no] = review_date
        
        for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
            if comment["c_idx"] == review_no:
                if comment.find("span", attrs={"class" : "cmt_summary"}) is not None:
                    verifying_text = comment.find("span", attrs={"class" : "cmt_summary"}).text
                    p = re.compile("안녕하세요\D+무신사\D+")
                    o = re.compile("지급해드렸습니다")
                    if p.search(verifying_text) is not None:
                        if o.search(verifying_text) is None:
                            if reduce(lambda x,y: x+y, review_no.split('.'))\
                                > reduce(lambda x,y: x+y, comment.find("span", attrs={"class":"recomment_date"}).text[:10].split('.')):
#                                 print(verifying_text, review_no, review_date)
                                recomment_text = comment.find("span", attrs={"class":"cmt_detail"}).text
                                musinsa_cmt_list.append(recomment_text)
                                review_recomment_dict[review_no] = recomment_text
            
        
        # 리뷰 사진을 뽑는다
        for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
            review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
            review_image_url_list = list()
            if idx != idx_img:
                pass
            elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
                for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
                    if review_img_pre.find('img')["src"].startswith("//"):
                        review_img_url = "https:" + review_img_pre.find('img')["src"][:-5]
                    review_image_url_list.append(review_img_url)
#                 print(review_no, review_image_url_list)
                review_image_dict[review_no] = review_image_url_list

    

                       

print(review_image_dict)
print(review_date_dict)
print(review_recomment_dict)


{'16855270': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_60a28f06adafb.jpg'], '29266622': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_62a491756318f.jpg'], '17350718': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_60bf8ebb13ea9.jpg'], '17002917': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_60acaa086a37c.jpg'], '16559188': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_608e7a1949862.jpg'], '16408243': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_60840f36cc9b8.jpg'], '11093364': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_5f2fbd68a2faf.jpg'], '32143140': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_63187898403c2.jpg'], '30911171': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_62e409a91f4a9.jpg'], '30723306': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_62dd6e957ce82.jpg'], '30139611': ['https://image.msscdn.net/data/estimate/1446775_0/gallery_62c5ac1d

In [26]:
set(musinsa_cmt_list), len(set(musinsa_cmt_list)), len(musinsa_cmt_list), len(review_no_list), len(review_date_list), len(review_image_dict.keys())

({'안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 스타일 후기 적립금 지급 기준이 충족되지 않아 지급이 보류되었습니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리겠습니다.▶ 스타일 후기 적립금 지급 조건- 해당 상품을 착용한 전신 정면 스타일링(어깨부터 발끝까지 모두 나오는) 사진- 착용이 어려운 상품은 해당 상품이 잘 보이도록 휴대 또는 함께 있는 전신 사진 - 20자 이상의 상품에 대한 평가(반복된 글자, 관련없는 내용은 X)▶ 상품평 수정 방법- 마이페이지 → 후기 작성 → 후기 내역(수정 희망하시는 후기 내용 클릭) → 수정/삭제※ 수정된 후기의 검수 및 적립금 지급까지는 평일 기준 1~2일 정도 소요됩니다.※ 댓글은 관리자가 확인하지 않습니다. 별도 문의사항은 1:1문의 작성을 부탁드립니다.',
  '안녕하세요. 무신사 스토어 고객센터입니다. 정성스럽게 올려주신 소중한 후기 감사합니다.그러나 현재 적립금 지급 기준이 충족되지 않아 적립금 지급이 보류된 상태입니다.수고로우시겠지만, 아래 기준을 참고하여 수정해 주시면 재검토 후 적립금 지급 도와드리도록 하겠습니다. 지급 보류 유형 8.전신 실루엣 식별 어려운 컷▶ 지급 보류 상세 사유 : 신체 일부가 가려진 상태의 사진 컷 사용TIP. 얼굴은 제외 가능하며, 어깨부터 발끝까지 프레임 안에 담겨야 합니다.착용이 어려운 상품은 휴대 상태 전신 컷 또는 전신 스타일링 컷 1장, 구매하신 상품 단독 컷 1장으로 올려주시면 적립금 지급이 가능합니다.(예:이너웨어, 화장품, 스마트기기, 옷걸이 등등)특수기호(~!@) 또는 반복되는 글자나 후기와 관련 없는 내용은 카운트되지 않습니다.▶ 스타일 후기 적립금 지급 조건 - 반드시 구매하신 상품을 착용한 전신 스타일링 컷- 착용이 어려운 상품은 전신 스타일링한 상태에서 해당 상품이 보이도록 든 상태의 전신 컷 - 20자 이상의 상품에 대한 후기 ▶ 상품평 수정 방법- PC, APP 

In [112]:
import sqlite3

In [113]:
con = sqlite3.connect("./musinsa_1101.db")
sql = "SELECT goods_id FROM 리뷰개수테이블 WHERE stylereviewCount >= 50 AND photoreviewCount >= 50;"
cur = con.cursor()
cur.execute(sql)
rows = cur.fetchall()
con.close()

In [114]:
goodcode_list = list()
for i in rows:
    goodcode_list.append(i[0])

In [115]:
len(goodcode_list)

5191

In [121]:
con = sqlite3.connect("./musinsa_1101.db")
sql = "SELECT * FROM 리뷰개수테이블 WHERE stylereviewCount >= 50 AND photoreviewCount >= 50;"
cur = con.cursor()
cur.execute(sql)
tuples = cur.fetchall()
con.close()

In [122]:
tuples

[(0, '947067', 17553, 2025, 3149),
 (1, '1208839', 17801, 2373, 3460),
 (2, '897632', 13611, 1441, 2356),
 (3, '1557508', 13548, 2124, 2974),
 (4, '956939', 11631, 1881, 2424),
 (5, '2038497', 11679, 1831, 2190),
 (6, '836499', 9171, 935, 1563),
 (7, '1582356', 11115, 1809, 2345),
 (8, '836495', 8724, 810, 1437),
 (9, '643173', 8467, 1230, 1544),
 (10, '1160367', 8520, 1171, 1989),
 (11, '836498', 7437, 901, 1382),
 (12, '1336836', 7833, 946, 1456),
 (13, '644037', 6747, 658, 1191),
 (14, '620634', 5903, 624, 1088),
 (15, '454425', 5826, 568, 939),
 (16, '2085371', 6903, 1072, 1238),
 (17, '1679943', 6668, 1135, 1364),
 (18, '947068', 5774, 712, 1154),
 (19, '1755609', 5544, 834, 1167),
 (20, '1126435', 4899, 625, 1018),
 (21, '1563623', 5065, 680, 1070),
 (22, '1557509', 4906, 729, 1048),
 (23, '1129701', 4216, 395, 835),
 (24, '1817394', 4752, 669, 828),
 (25, '1642887', 4483, 659, 881),
 (26, '1177545', 3819, 566, 790),
 (27, '1549281', 4335, 716, 1202),
 (28, '1759350', 4302, 730, 

In [312]:
tuples[0]

(0, '947067', 17553, 2025, 3149)

In [313]:
tuples[1]

(1, '1208839', 17801, 2373, 3460)

In [315]:
### 비율에 맞게 가중치, ###
### DB 에 쓸 코드임 ### 

url_good =  "https://goods.musinsa.com/api/goods/v1/review/style/list"
params = {
    'page' : None,
    'goodsNo' : None,
    'sort' : "comment_cnt_desc"
    #'similarNo' : None, # goodsNo와 동일
    # 'selectedSimilarNo' : None # 색상 아닌 전체
    }

musinsa_cmt_list = list()
review_no_list = list()
review_no_dict = dict()
review_date_list = list()
review_date_dict = dict() # 키값이 리뷰넘버, 값이 스트링
review_image_dict = dict() # 키값이 리뷰넘버, 값이 url 리스트
review_recomment_dict = dict()
goods_list = list()
review_image_num_dict = dict()

for index in range(len(tuples[:2])):
    goodsNo = tuples[index][1]
    search_pages = tuples[index][3]//40
    print(goodsNo, search_pages)
    for i in range(search_pages): # 페이지 range을 수정해야한다. 
        params['page'] = i+1
        params['goodsNo'] = goodsNo
        # params['similarNo'] = goodsNo
        # params['selectedSimilarNo'] = 0
        res = requests.get(url_good, params = params, headers=header)
        soup = BeautifulSoup(res.text, "html.parser")

            # 페이지마다 모두 뽑기
        comments = soup.find_all(attrs={"class":"comments comments-input"})
        reviews = soup.find_all(attrs={"class":"review-contents"})
        review_dates = soup.find_all("p", attrs={"class":"review-profile__date"})
        review_images = soup.find_all('div', attrs={"class" : "review-content-photo"})
#         print("상품", goodsNo)


            # 리뷰 넘버를 뽑는다
        for idx,review in enumerate(reviews):
            review_no = review["data-review-no"]
            review_no_list.append(review_no)
            goods_list.append(goodsNo)
            review_no_dict[goodsNo] = review_no_list 
#             print("리뷰넘버 :", review_no)
                # 리뷰 데이트를 뽑는다    

            for idx_date, date in enumerate(review_dates):
                if idx != idx_date:
                    pass
                elif idx == idx_date:
                    if len(date.text.split('.')) ==3:
                        review_date = date.text
                    else:
                        review_date = "며칠전으로 표시된 리뷰"
            review_date_list.append(review_date)
            review_date_dict[review_no] = review_date
            
print(review_no_dict, review_date_dict)



#             for idx_cmt, comment in enumerate(comments): # 각 리뷰에 달린 무신사측 답글을 고르기
# #                 print(idx_cmt)
#                 if idx != idx_cmt:
#                     pass
#                 elif idx == idx_cmt:
#                     find_recomments = comment.find_all("div", attrs={"class":"comment_recomment comment_wrap"}, recursive=True)
#                     for idx_rcmt, find_recomment in enumerate(find_recomments):
# #                         print("답글번호", idx_rcmt, find_recomment.find("span", attrs={"class":"cmt_summary"}))
#                         if find_recomment.find("img", attrs={"alt":"무신사 관리자 썸네일"}) is not None: # 무신사가 답글 단 경우 
#                             recomment_sum = find_recomment.find("span", attrs={"class":"cmt_summary"})
#                             recomment_date = find_recomment.find("span", attrs={"class":"recomment_date"})
#                             recomment_detail = find_recomment.find("span", attrs={"class":"cmt_detail"})
# #                             print(recomment_sum, recomment_date, recomment_detail)
# #                             print(review_date_dict[comment["c_idx"]].split('.'), recomment_date.text[:10].split('.'))

#                             if reduce(lambda x,y: x+y, review_date_dict[comment["c_idx"]].split('.'))\
#                                         > reduce(lambda x,y: x+y, recomment_date.text[:10].split('.')): # 리뷰날짜가 답글날짜보다 최신이면 정상
#                                 pass
#                             else:
#                                 if recomment_detail:
#                                     verifying_text = recomment_detail.text
#                                     p = re.compile("안녕하세요\D+무신사\D+")
#                                     o = re.compile("지급해드렸습니다")
#                                     if p.search(verifying_text) is not None:
#                                         if o.search(verifying_text) is None: # 지급해드렸습니다 답글 걸러냄
# #                                             print(review_date_dict[comment["c_idx"]], recomment_date.text[:10], recomment_detail.text)

#                                             musinsa_cmt_list.append(recomment_detail.text)
#                                             review_recomment_dict[review_no] = recomment_detail.text

# #             print(len(review_recomment_dict.keys()) )


#              # 리뷰 사진을 뽑는다
#             for idx_img, review_image_source in enumerate(review_images): # 10개의 리뷰
#                 review_img_pres = review_image_source.find_all("li", attrs={'class':'review-content-photo__item'})
#                 review_image_url_list = list()
#                 if idx != idx_img:
#                     pass
#                 elif idx == idx_img: # 리뷰와 리뷰이미지 순서 매치
#                     for idx_img_img, review_img_pre in enumerate(review_img_pres): # 한 리뷰당 복수의 이미지
#                         if review_img_pre.find('img')["src"].startswith("//"):
#                             review_img_url = ("https:" + review_img_pre.find('img')["src"][:-5])
#                         review_image_url_list.append(review_img_url)
#         #                 print(review_no, review_image_url_list)
#                     review_image_dict[review_no] = review_image_url_list

#                     review_image_num_dict[review_no] = len(review_image_url_list)



# print(len(review_image_dict.keys()))

# print(len(review_date_dict.keys()))

# print(len(review_recomment_dict.keys()))

# print(review_image_dict)

# print(review_date_dict)

# print(review_recomment_dict.keys())

# print("상품", goodsNo)

947067 50
1208839 59
{'947067': ['7997463', '4067306', '8468969', '7182239', '7001487', '4026710', '32574067', '22645244', '22302492', '21997506', '21877078', '16297939', '15146783', '15129688', '14623781', '14187792', '14053827', '13795171', '13406286', '13348723', '13137984', '13056054', '12963421', '12429895', '12428208', '12198132', '12158502', '12092140', '12062653', '12026394', '11688662', '11643797', '11438313', '9360715', '9107777', '9085396', '8902792', '8824616', '8815192', '8814458', '8768914', '8596615', '8730965', '8741747', '8741602', '8730858', '8473662', '8356576', '8350334', '8260396', '6231471', '8225471', '8036030', '8093861', '8047983', '7817657', '7778367', '7669271', '7647174', '7545752', '7637723', '7409211', '7525087', '7493449', '7422201', '7373571', '7369238', '7359442', '7342370', '7333588', '7330226', '7324257', '7282165', '7227469', '7061438', '7017198', '7014941', '6998593', '6998456', '6986205', '6975995', '6959429', '6940773', '6919206', '6851591', '6851

In [267]:
print(review_no_dict)

{'947067': ['7997463', '4067306', '8468969', '7182239', '7001487', '4026710', '32574067', '22645244', '22302492', '21997506', '21877078', '16297939', '15146783', '15129688', '14623781', '14187792', '14053827', '13795171', '13406286', '13348723', '13137984', '13056054', '12963421', '12429895', '12428208', '12198132', '12158502', '12092140', '12062653', '12026394', '11688662', '11643797', '11438313', '9360715', '9107777', '9085396', '8902792', '8824616', '8815192', '8814458', '8768914', '8596615', '8730965', '8741747', '8741602', '8730858', '8473662', '8356576', '8350334', '8260396', '6231471', '8225471', '8036030', '8093861', '8047983', '7817657', '7778367', '7669271', '7647174', '7545752', '7637723', '7409211', '7525087', '7493449', '7422201', '7373571', '7369238', '7359442', '7342370', '7333588', '7330226', '7324257', '7282165', '7227469', '7061438', '7017198', '7014941', '6998593', '6998456', '6986205', '6975995', '6959429', '6940773', '6919206', '6851591', '6851588', '6813350', '679

In [268]:
print(review_image_dict)

print(review_date_dict)

print(review_recomment_dict.keys())
 

{'7997463': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5e25e7cf06ad3.jpg'], '4067306': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5c76931b8e296.jpg'], '8468969': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5e58df0be27b7.jpg'], '7182239': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5dd51c4c4d0c2.jpg'], '7001487': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5dc5235d5576f.jpg'], '4026710': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5c7c95f6bb701.jpg'], '32574067': ['https://image.msscdn.net/data/estimate/947067_0/gallery_6328021b0bf17.jpg'], '22645244': ['https://image.msscdn.net/data/estimate/947067_0/gallery_61a5e38a53dc8.jpg'], '22302492': ['https://image.msscdn.net/data/estimate/947067_0/gallery_619d9a27a301b.jpg', 'https://image.msscdn.net/data/estimate/947067_0/gallery_619d9a285e088.jpg'], '21997506': ['https://image.msscdn.net/data/estimate/947067_0/gallery_619349e0adc2a.jpg'], '21877078': ['http

In [269]:
con.close()

In [270]:
con = sqlite3.connect("./musinsa_1101.db")
cur = con.cursor()
cur.executescript('''
    DROP TABLE IF EXISTS 리뷰테이블;
    CREATE TABLE 리뷰테이블(
        IDX INTEGER PRIMARY KEY AUTOINCREMENT,    
        goodsId INTEGER,
        reviewId INTEGER
    );
    
    DROP TABLE IF EXISTS 리뷰이미지;
    CREATE TABLE 리뷰이미지(
        reviewId INTEGER,
        reviewImageUrl,
        reviewImageNum INTEGER
    );

    DROP TABLE IF EXISTS 리뷰날짜;
    CREATE TABLE 리뷰날짜(
        reviewId INTEGER,
        reviewDate TEXT        
    );
    
    DROP TABLE IF EXISTS 리뷰답글;
    CREATE TABLE 리뷰답글(
        reviewId INTEGER,
        reviewRecomment TEXT
    );
''')
con.commit()


In [290]:
review_no_dict

{'947067': ['7997463',
  '4067306',
  '8468969',
  '7182239',
  '7001487',
  '4026710',
  '32574067',
  '22645244',
  '22302492',
  '21997506',
  '21877078',
  '16297939',
  '15146783',
  '15129688',
  '14623781',
  '14187792',
  '14053827',
  '13795171',
  '13406286',
  '13348723',
  '13137984',
  '13056054',
  '12963421',
  '12429895',
  '12428208',
  '12198132',
  '12158502',
  '12092140',
  '12062653',
  '12026394',
  '11688662',
  '11643797',
  '11438313',
  '9360715',
  '9107777',
  '9085396',
  '8902792',
  '8824616',
  '8815192',
  '8814458',
  '8768914',
  '8596615',
  '8730965',
  '8741747',
  '8741602',
  '8730858',
  '8473662',
  '8356576',
  '8350334',
  '8260396',
  '6231471',
  '8225471',
  '8036030',
  '8093861',
  '8047983',
  '7817657',
  '7778367',
  '7669271',
  '7647174',
  '7545752',
  '7637723',
  '7409211',
  '7525087',
  '7493449',
  '7422201',
  '7373571',
  '7369238',
  '7359442',
  '7342370',
  '7333588',
  '7330226',
  '7324257',
  '7282165',
  '7227469',
 

In [300]:
len(review_no_dict['947067'])

1090

In [302]:
len(review_no_dict['1208839'])

1090

In [303]:
review_no_dict['947067'] == review_no_dict['1208839']

True

In [297]:
db_review_no_list = [(list(review_no_dict.keys())[v] ,list(review_no_dict.values())[v][k])\
                     for v in range(len(list(review_no_dict.keys())))\
                      for k in range(len(list(review_no_dict.values())[v]))]


In [301]:
db_review_no_list[1090]

('1208839', '7997463')

In [273]:
cur.executemany('''
    INSERT INTO 리뷰테이블(goodsId, reviewId)
    VALUES(?, ?)
''', db_review_no_dict)

In [274]:
con.commit()

In [287]:
review_image_dict

{'7997463': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5e25e7cf06ad3.jpg'],
 '4067306': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5c76931b8e296.jpg'],
 '8468969': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5e58df0be27b7.jpg'],
 '7182239': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5dd51c4c4d0c2.jpg'],
 '7001487': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5dc5235d5576f.jpg'],
 '4026710': ['https://image.msscdn.net/data/estimate/947067_0/gallery_5c7c95f6bb701.jpg'],
 '32574067': ['https://image.msscdn.net/data/estimate/947067_0/gallery_6328021b0bf17.jpg'],
 '22645244': ['https://image.msscdn.net/data/estimate/947067_0/gallery_61a5e38a53dc8.jpg'],
 '22302492': ['https://image.msscdn.net/data/estimate/947067_0/gallery_619d9a27a301b.jpg',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_619d9a285e088.jpg'],
 '21997506': ['https://image.msscdn.net/data/estimate/947067_0/gallery_619349e0adc2a.jpg'],
 '21877

In [278]:
list(review_image_dict.keys())[1]

'4067306'

In [283]:
list(review_image_dict.values())[1][0]

'https://image.msscdn.net/data/estimate/947067_0/gallery_5c76931b8e296.jpg'

In [284]:
db_review_image_dict = [(list(review_image_dict.keys())[v] ,list(review_image_dict.values())[v][k]) for k in range(len(list(review_image_dict.values())[0])) \
                     for v in range(len(list(review_image_dict.keys())))]

In [285]:
db_review_image_dict

[('7997463',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5e25e7cf06ad3.jpg'),
 ('4067306',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5c76931b8e296.jpg'),
 ('8468969',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5e58df0be27b7.jpg'),
 ('7182239',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5dd51c4c4d0c2.jpg'),
 ('7001487',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5dc5235d5576f.jpg'),
 ('4026710',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_5c7c95f6bb701.jpg'),
 ('32574067',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_6328021b0bf17.jpg'),
 ('22645244',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_61a5e38a53dc8.jpg'),
 ('22302492',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_619d9a27a301b.jpg'),
 ('21997506',
  'https://image.msscdn.net/data/estimate/947067_0/gallery_619349e0adc2a.jpg'),
 ('21877078',
  'https://image.msscdn.net/data/estimate/947067_0/g

In [ ]:
cur.executemany('''
    INSERT INTO 리뷰이미지(, reviewId)
    VALUES(?, ?, ?)
''', db_review_no_dict)

In [ ]:
        reviewImageUrl,
        reviewImageNum INTEGER,
        reviewDate TEXT,
        reviewRecomment TEXT